In [ ]:
import math
import random
from collections.abc import Iterable, MutableMapping
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import sortedcontainers
from pandas.io.formats.style import Styler

import pysorteddict

sorted_dict_lens = [100, 1_000, 10_000, 100_000, 1_000_000, 10_000_000]
sorted_dict_types = [pysorteddict.SortedDict, sortedcontainers.SortedDict]


def setup(sorted_dict_type: type, sorted_dict_len: int, seed: float = math.pi) -> MutableMapping:
    random.seed(seed)
    d = sorted_dict_type()
    for _ in range(sorted_dict_len):
        d[random.random()] = random.random()
    return d


def plot(stylers: Iterable[Styler], *, alpha: float = 0.7, logy: bool = False, name: str = "", yunit: str = "s"):
    for theme in ["light", "dark"]:
        with plt.style.context(f"{theme}.mplstyle", after_reset=True):
            fig, ax = plt.subplots()
            for styler, marker, loc in zip(stylers, ["D", "s"], ["upper left", "upper center"], strict=False):
                ax_lines_len = len(ax.lines)
                ax.set_prop_cycle(None)
                styler.data.plot(
                    alpha=alpha,
                    ax=ax,
                    logx=True,
                    logy=logy,
                    marker=marker,
                    xlabel="Sorted Dictionary Length",
                    xlim=(sorted_dict_lens[0] // 10, sorted_dict_lens[-1] * 10),
                    ylabel="Average Execution Time",
                )
                # Create a separate legend for these lines.
                font_prop = {"family": "JetBrains Mono"}
                ax.add_artist(
                    ax.legend(
                        handles=ax.lines[ax_lines_len:], loc=loc, prop=font_prop, title=styler.caption, title_fontproperties=font_prop,
                    ),
                )
            # Legends have been added manually. Remove the automatically added one.
            ax.get_legend().remove()
            ax.grid(which="major", linewidth=0.5, linestyle=":")
            ax.grid(which="minor", axis="y", linewidth=0.0625, linestyle="-")
            ax.minorticks_on()
            # The warning about setting labels without setting ticks can be ignored because the plot isn't interactive.
            ax.set_yticklabels(f"{label.get_text()} {yunit}" for label in ax.get_yticklabels())
            plt.show()
            fig.savefig(Path().parents[1] / "docs" / "_static" / "images" / f"perf-{name}-{theme}.svg")

# `contains`

In [ ]:
def bench_contains() -> Iterable[Styler]:
    for sorted_dict_type in sorted_dict_types:
        df = pd.DataFrame(index=sorted_dict_lens, columns=["0.00 in d", "0.33 in d", "0.67 in d", "1.00 in d"])
        for i, sorted_dict_len in enumerate(sorted_dict_lens):
            d = setup(sorted_dict_type, sorted_dict_len)
            for j, key in enumerate([0.00, 0.33, 0.67, 1.00]):
                %timeit -v bench key in d
                df.iat[i, j] = bench.average * 1e9
        yield df.style.set_caption(sorted_dict_type.__module__)


stylers = [*bench_contains()]

In [ ]:
for styler in stylers:
    display(styler)
plot(stylers, name="contains", yunit="ns")

# `setitem`

In [ ]:
def set_del(d: MutableMapping, keys: list[float]):
    for key in keys:
        d[key] = None
    for key in keys:
        del d[key]


def bench_setitem() -> Iterable[Styler]:
    keys_lens = [33, 67, 100]
    for sorted_dict_type in sorted_dict_types:
        df = pd.DataFrame(index=sorted_dict_lens, columns=[f"set_del(d, keys_{keys_len})" for keys_len in keys_lens])
        for i, sorted_dict_len in enumerate(sorted_dict_lens):
            for j, keys_len in enumerate(keys_lens):
                d = setup(sorted_dict_type, sorted_dict_len)
                keys = [random.random() for _ in range(keys_len)]
                %timeit -v bench set_del(d, keys)
                df.iat[i, j] = bench.average * 1e6
        yield df.style.set_caption(sorted_dict_type.__module__)


stylers = [*bench_setitem()]

In [ ]:
for styler in stylers:
    display(styler)
plot(stylers, name="setitem", yunit="μs")

# `iter`

In [ ]:
def bench_iter() -> Iterable[Styler]:
    for sorted_dict_type in sorted_dict_types:
        df = pd.DataFrame(index=sorted_dict_lens, columns=["for _ in d: pass", "for _ in reversed(d): pass"])
        for i, sorted_dict_len in enumerate(sorted_dict_lens):
            d = setup(sorted_dict_type, sorted_dict_len)
            %timeit -v bench for _ in d: pass
            df.iat[i, 0] = bench.average
            %timeit -v bench for _ in reversed(d): pass
            df.iat[i, 1] = bench.average
        yield df.style.set_caption(sorted_dict_type.__module__)


stylers = [*bench_iter()]

In [ ]:
for styler in stylers:
    display(styler)
plot(stylers, logy=True, name="iter")